In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
data_2 = pd.read_csv(r'diabetes.csv')
data_2

data_2=data_2.fillna(0) # fill NAs  with 0 

X = data_2.drop(['Outcome'], axis=1)
y = data_2.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [3]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
# Select an algorithm
algorithm = KNeighborsClassifier()
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)
# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]
# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
knn =grid.fit(X_train, y_train)

# Get the results
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_) 
# Create 3 folds

knn_predictions = knn.predict(X_test) 
cm = confusion_matrix(y_test, knn_predictions)
print(cm)
classification_report(y_test, knn_predictions)
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print(accuracy)

-0.16806331659066576
KNeighborsClassifier(n_neighbors=6, weights='distance')
{'n_neighbors': 6, 'weights': 'distance'}
[[137  27]
 [ 45  45]]
-0.23902439024390243


In [4]:
#Naïve Bayes
from sklearn.naive_bayes import GaussianNB
gnb_clf=GaussianNB()
parameters = {
    'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]
}
clf = GridSearchCV(gnb_clf, parameters, cv=5)
gnb=clf.fit(X_train, y_train)


gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(cm)
cls_report=classification_report(y_test, gnb_predictions)
print(cls_report)

0.7598425196850394
[[143  21]
 [ 40  50]]
              precision    recall  f1-score   support

           0       0.78      0.87      0.82       164
           1       0.70      0.56      0.62        90

    accuracy                           0.76       254
   macro avg       0.74      0.71      0.72       254
weighted avg       0.75      0.76      0.75       254



In [12]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
model = LogisticRegression(solver='liblinear')

c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=1, cv=cv)
grid_result = grid_search.fit(X_train, y_train)


logreg=model.fit(X_train, y_train)
log_predictions = model.predict(X_test)
print(log_predictions)
# accuracy on X_test
accuracy = grid_result.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, log_predictions)
print(cm)
cl_repot=classification_report(y_test, log_predictions)
print(cl_repot)

[0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
0.7755905511811023
[[150  14]
 [ 41  49]]
              precision    recall  f1-score   support

           0       0.79      0.91      0.85       164
           1       0.78      0.54      0.64        90

    accuracy                           0.78       254
   macro avg       0.78      0.73      0.74       254
weighted avg       0.78      0.78      0.77       254



In [13]:
import pickle
with open('model_diabits','wb') as file:
    pickle.dump(gnb,file)

In [14]:
with open('model_diabits','rb') as file:
    mp = pickle.load(file)
    
pre=mp.predict(X_test)
print(pre) 

[0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1
 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0]


In [6]:
#Decision Tree algorithms
from sklearn.tree import DecisionTreeClassifier
model= DecisionTreeClassifier()
parameters={
            "max_depth" : [7,9,11,12,15],
           "min_samples_leaf":[50,10,100],
           "min_weight_fraction_leaf":[0.0001,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_leaf_nodes":[None,1,20,30,40,50,60,70,80,90] }

tuning_model=GridSearchCV(estimator=model,param_grid=parameters,cv=3,verbose=3)

dtree_model =tuning_model.fit(X_train, y_train)

#dtree_model = DecisionTreeClassifier(max_depth = 15).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
accuracy = dtree_model.score(X_test, y_test)
print(accuracy)  
# creating a confusion matrix
cm = confusion_matrix(y_test, dtree_predictions)
print(cm)

cl_report=classification_report(y_test, dtree_predictions)
print(cl_report)

Fitting 3 folds for each of 1350 candidates, totalling 4050 fits
[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.0001;, score=0.738 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.0001;, score=0.743 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.0001;, score=0.713 total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.2;, score=0.738 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.2;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.2;, score=0.754 total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=50, min_weight_fraction_leaf=0.3;, score=0.727 total time=   0.0s
[CV 2/3] END max

[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=0.651 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=0.655 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=0.655 total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_leaf=0.7;, score=nan total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=None, min_samples_leaf=100, min_weight_fraction_le

[CV 3/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.4;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=nan total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=nan total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.5;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=1, min_samples_leaf=100, min_weight_fraction_leaf=0.7;, score=nan total time=

[CV 2/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=10, min_weight_fraction_leaf=0.8;, score=nan total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=10, min_weight_fraction_leaf=0.8;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=20, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score

[CV 2/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.3;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.3;, score=0.743 total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.4;, score=0.622 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.4;, score=0.673 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.4;, score=0.737 total time=   0.0s
[CV 1/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.5;, score=0.651 total time=   0.0s
[CV 2/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.5;, score=0.655 total time=   0.0s
[CV 3/3] END max_depth=7, max_leaf_nodes=30, min_samples_leaf=10, min_weight_fraction_leaf=0.5;, score=0

...

[CV 1/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.743 total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.2;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.2;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.2;, score=0.743 total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fraction_leaf=0.3;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=60, min_samples_leaf=100, min_weight_fr

[CV 2/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.743 total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction_leaf=0.2;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction_leaf=0.2;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=70, min_samples_leaf=100, min_weight_fraction

[CV 2/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=10, min_weight_fraction_leaf=0.8;, score=nan total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=10, min_weight_fraction_leaf=0.8;, score=nan total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=10, min_weight_fraction_leaf=0.9;, score=nan total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.727 total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=100, min_weight_fraction_leaf=0.0001;, score=0.708 total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=80, min_samples_leaf=100, min_weight_fraction_leaf=0.0001

[CV 3/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.5;, score=0.655 total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.6;, score=nan total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.7;, score=nan total time=   0.0s
[CV 2/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.7;, score=nan total time=   0.0s
[CV 3/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.7;, score=nan total time=   0.0s
[CV 1/3] END max_depth=15, max_leaf_nodes=90, min_samples_leaf=10, min_weight_fraction_leaf=0.8;, score=nan t

/home/dilum/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2025 fits failed out of a total of 4050.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1800 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dilum/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dilum/.local/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/home/dilum/.local/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min